In [ ]:
import os
import glob
import time
import random
import numpy as np
import cv2

FOLDER = #Folder with 30 images
OUTDIR = os.path.join(FOLDER, "")

K_FIXED = 0.00128633
YB = 0.0
YM = 245.0

XB_Q = 90.0
XM_Q = 70.0

SAMPLE_N = 30
RANDOM_SEED = 2026 

SCALE = 0.25
PROGRESS_STEP = 10

MEDIAN_KSIZE = 5
P_SEED = 97.0
P_LOW = 60.0
DILATE_PX = 15
CLOSE_PX = 0

def list_images_recursive(folder):
    exts = ("*.png", "*.jpg", "*.jpeg", "*.bmp", "*.tif", "*.tiff")
    files = []
    for e in exts:
        files.extend(glob.glob(os.path.join(folder, "**", e), recursive=True))
    return sorted(files)

def to_gray(img):
    if img is None:
        return None
    if len(img.shape) == 2:
        return img
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def ensure_u8(gray):
    if gray is None:
        return None
    if gray.dtype == np.uint8:
        return gray
    if gray.dtype == np.uint16:
        return (gray / 256).astype(np.uint8)
    return cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

def maybe_resize(gray_u8, scale):
    if scale is None or abs(scale - 1.0) < 1e-9:
        return gray_u8
    h, w = gray_u8.shape[:2]
    new_w = max(1, int(w * scale))
    new_h = max(1, int(h * scale))
    return cv2.resize(gray_u8, (new_w, new_h), interpolation=cv2.INTER_AREA)

def largest_connected_component(binary_u8):
    bin01 = (binary_u8 > 0).astype(np.uint8)
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(bin01, connectivity=8)
    if num_labels <= 1:
        return np.zeros_like(binary_u8, dtype=np.uint8)
    areas = stats[1:, cv2.CC_STAT_AREA]
    max_idx = 1 + int(np.argmax(areas))
    return (labels == max_idx).astype(np.uint8) * 255

def build_masks_two_stage(I_u8, p_seed=97.0, p_low=60.0, dilate_px=15, close_px=0):
    T_seed = np.percentile(I_u8, p_seed)
    seed = (I_u8 >= T_seed).astype(np.uint8) * 255

    if close_px and close_px > 0:
        k = int(close_px)
        if k % 2 == 0:
            k += 1
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k, k))
        seed = cv2.morphologyEx(seed, cv2.MORPH_CLOSE, kernel, iterations=1)

    seed = largest_connected_component(seed)
    seed01 = (seed > 0)

    T_low = np.percentile(I_u8, p_low)
    cand01 = (I_u8 >= T_low).astype(np.uint8)

    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(cand01, connectivity=8)
    if num_labels <= 1:
        fg = seed.copy()
    else:
        best_label = 0
        best_overlap = -1
        for lab in range(1, num_labels):
            comp = (labels == lab)
            overlap = int(np.sum(comp & seed01))
            if overlap > best_overlap:
                best_overlap = overlap
                best_label = lab
        fg = (labels == best_label).astype(np.uint8) * 255 if best_label != 0 else seed.copy()

    if dilate_px and dilate_px > 0:
        k = int(dilate_px)
        if k % 2 == 0:
            k += 1
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k, k))
        fg_d = cv2.dilate(fg, kernel, iterations=1)
    else:
        fg_d = fg.copy()

    bg = (fg_d == 0).astype(np.uint8) * 255
    return fg, bg


def progress_report(i, n, t0, last_pct_reported):
    pct = int((i + 1) * 100 / n)
    if pct >= last_pct_reported + PROGRESS_STEP or pct == 100:
        elapsed = time.time() - t0
        rate = elapsed / max(1, i + 1)
        eta = rate * (n - (i + 1))
        print(f"[PROGRESS] {pct}% ({i+1}/{n})  elapsed={elapsed:.1f}s  ETA≈{eta:.1f}s")
        return pct
    return last_pct_reported


def solve_m_c_fixed_k(xb, xm, yb, ym, k):
    denom = np.exp(k * xm) - np.exp(k * xb)
    if abs(denom) < 1e-12:
        raise RuntimeError(
            f"Denominator too small: exp(k*xm)≈exp(k*xb). xb={xb:.3f}, xm={xm:.3f}."
        )
    m = (ym - yb) / denom
    c = yb - m * np.exp(k * xb)
    return float(m), float(c)

def apply_transform(I_u8, m, k, c):
    g = m * np.exp(k * I_u8.astype(np.float32)) + c
    g = np.clip(g, 0, 255).astype(np.uint8)
    return g

def main():
    all_files = list_images_recursive(FOLDER)
    if len(all_files) == 0:
        raise FileNotFoundError(f"No images found in directory: {FOLDER}")

    random.seed(RANDOM_SEED)
    if len(all_files) <= SAMPLE_N:
        files = all_files
    else:
        files = random.sample(all_files, SAMPLE_N)
        files = sorted(files)

    n = len(files)

    print(f"[INFO] Total images found: {len(all_files)}")
    print(f"[INFO] Random-sampled images used: {n} (SAMPLE_N={SAMPLE_N}, random seed={RANDOM_SEED})")
    print(f"[INFO] Image scale factor: {SCALE}")
    print(f"[INFO] Fixed k value={K_FIXED}, Target YB={YB}, Target YM={YM}")
    print(f"[INFO] Anchor percentiles: xb=P{XB_Q}(BG), xm=P{XM_Q}(FG), cross-frame median will be calculated")

    ksize = MEDIAN_KSIZE if MEDIAN_KSIZE % 2 == 1 else MEDIAN_KSIZE + 1

    xb_i_list, xm_i_list = [], []
    records = []
    valid = 0

    print("\n[STAGE 1] Calculating per-frame xb_i/xm_i anchors ...")
    t0 = time.time()
    last_pct = 0

    for i, fp in enumerate(files):
        img = cv2.imread(fp, cv2.IMREAD_UNCHANGED)
        if img is None:
            last_pct = progress_report(i, n, t0, last_pct)
            continue

        gray = ensure_u8(to_gray(img))
        gray = maybe_resize(gray, SCALE)
        I = cv2.medianBlur(gray, ksize)

        fg, bg = build_masks_two_stage(I, p_seed=P_SEED, p_low=P_LOW,
                                       dilate_px=DILATE_PX, close_px=CLOSE_PX)

        fg_pixels = I[fg > 0]
        bg_pixels = I[bg > 0]
        if fg_pixels.size < 200 or bg_pixels.size < 200:
            last_pct = progress_report(i, n, t0, last_pct)
            continue

        xb_i = float(np.percentile(bg_pixels, XB_Q))
        xm_i = float(np.percentile(fg_pixels, XM_Q))

        xb_i_list.append(xb_i)
        xm_i_list.append(xm_i)
        valid += 1
        records.append((os.path.basename(fp), xb_i, xm_i, float(np.mean(fg > 0))))

        last_pct = progress_report(i, n, t0, last_pct)

    if valid < max(10, int(0.6 * n)):
        raise RuntimeError(f"Insufficient valid frames for anchor statistics: valid={valid}/{n} (minimum required: {max(10, int(0.6 * n))}).")

    xb_bar = float(np.median(xb_i_list))
    xm_bar = float(np.median(xm_i_list))

    print("\n[RESULT] Stable cross-frame anchor values (median):")
    print(f"  x_b_bar = {xb_bar:.3f}")
    print(f"  x_m_bar = {xm_bar:.3f}")

    m, c = solve_m_c_fixed_k(xb_bar, xm_bar, YB, YM, K_FIXED)
    print("\n[RESULT] Calculated mkc parameters (fixed k):")
    print(f"  m = {m:.6f}")
    print(f"  k = {K_FIXED:.8f}")
    print(f"  c = {c:.6f}")

    # Validation check
    print("\n[STAGE 2] Performing validation checks (FG saturation & BG output statistics) ...")
    sat_list = []
    bg_med_list = []
    bg_p90_list = []
    t1 = time.time()
    last_pct = 0

    for i, fp in enumerate(files):
        img = cv2.imread(fp, cv2.IMREAD_UNCHANGED)
        if img is None:
            last_pct = progress_report(i, n, t1, last_pct)
            continue

        gray = ensure_u8(to_gray(img))
        gray = maybe_resize(gray, SCALE)
        I = cv2.medianBlur(gray, ksize)

        fg, bg = build_masks_two_stage(I, p_seed=P_SEED, p_low=P_LOW,
                                       dilate_px=DILATE_PX, close_px=CLOSE_PX)
        g = apply_transform(I, m, K_FIXED, c)

        fg_pixels = g[fg > 0]
        bg_pixels = g[bg > 0]
        if fg_pixels.size < 200 or bg_pixels.size < 200:
            last_pct = progress_report(i, n, t1, last_pct)
            continue

        sat_list.append(float(np.mean(fg_pixels >= 255)))
        bg_med_list.append(float(np.median(bg_pixels)))
        bg_p90_list.append(float(np.percentile(bg_pixels, 90)))

        last_pct = progress_report(i, n, t1, last_pct)

    if len(sat_list) > 0:
        print("\n[VALIDATION CHECK] FG saturation ratio: median=%.3f, p90=%.3f" %
              (float(np.median(sat_list)), float(np.percentile(sat_list, 90))))
        print("[VALIDATION CHECK] BG output grayscale values: median=%.2f, p90=%.2f" %
              (float(np.median(bg_med_list)), float(np.percentile(bg_p90_list, 90))))

    os.makedirs(OUTDIR, exist_ok=True)
    txt_path = os.path.join(OUTDIR, "mkc_fixedk_random30_result.txt")
    csv_path = os.path.join(OUTDIR, "anchors_per_frame_random30.csv")

    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(f"SOURCE_FOLDER={FOLDER}\n")
        f.write(f"Total_images_found={len(all_files)}\n")
        f.write(f"Sampled_images={n}, valid_frames={valid}, random_seed={RANDOM_SEED}\n")
        f.write(f"IMAGE_SCALE={SCALE}\n")
        f.write(f"Fixed_k_value={K_FIXED}\n")
        f.write(f"Target_anchor_values: YB={YB}, YM={YM}\n")
        f.write(f"Anchor_percentiles: xb_i=P{XB_Q}(BG), xm_i=P{XM_Q}(FG)\n")
        f.write(f"Stable_cross_frame_anchors: xb_bar={xb_bar:.6f}, xm_bar={xm_bar:.6f}\n")
        f.write(f"Calculated_transform_parameters: m={m:.6f}, c={c:.6f}\n")
        if len(sat_list) > 0:
            f.write(f"FG_saturation_ratio_median={float(np.median(sat_list)):.6f}, FG_saturation_ratio_p90={float(np.percentile(sat_list,90)):.6f}\n")
            f.write(f"BG_output_gray_median={float(np.median(bg_med_list)):.6f}, BG_output_gray_p90={float(np.percentile(bg_p90_list,90)):.6f}\n")

    with open(csv_path, "w", encoding="utf-8") as f:
        f.write("filename,xb_i_p90_bg,xm_i_p70_fg,fg_area_ratio\n")
        for name, xb_i, xm_i, fg_ratio in records:
            f.write(f"{name},{xb_i:.6f},{xm_i:.6f},{fg_ratio:.6f}\n")

    print(f"\n[COMPLETED] Results summary saved to: {txt_path}")
    print(f"[COMPLETED] Per-frame anchor values saved to: {csv_path}")


if __name__ == "__main__":
    main()